# TItanic Kaggle Compitation

In [9]:
import numpy as np #linear Algebra
import pandas as pd # data processing, CSV file I/O(eg. )
from subprocess import check_output

print(check_output(['ls', "input"]).decode("utf8"))

Titanic.csv
gender_submission.csv
pitanic.csv
test.csv
train.csv



In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#machine Learning
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import mean_squared_error, precision_score,precision_recall_curve,recall_score,f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.kernel_ridge import KernelRidge


In [10]:
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")

In [11]:
train.head().set_index("PassengerId")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [14]:
train.shape

(891, 12)

In [15]:
def ticket2(value):
    split = value.split(" ")
    if(len(split)>1):
        return int(split[-1].strip())
    else:
        return 0

In [16]:
dftotal=pd.concat([train,test],axis=0,ignore_index=True)
dftotal["Ticket1"]=dftotal.Ticket.map(lambda x: x.split(" ")[0])
dftotal["Ticket2"]=dftotal.Ticket.map(ticket2)
#dftotal["Ticket2"]=dftotal["Ticket2"].astype(int)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [17]:
dftotal.head()


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Ticket1,Ticket2
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,A/5,21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,PC,17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,STON/O2.,3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,113803,0
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,373450,0


In [18]:
dftotal["KNNs"]=""
dftotal["KNNsNbr"]=0
dftotal["KSurvivalRate"]=0

In [19]:
k=10
for index in dftotal.index:
    ticket1=dftotal.loc[index,"Ticket1"]
    ticket2=dftotal.loc[index,"Ticket2"]
    embarked = dftotal.loc[index,"Embarked"]
    if(ticket1=="LINE"):
        nbours_index=dftotal[dftotal.Ticket=="LINE"].index
        dftotal.at[index,"KNNs"]=nbours_index.values
        dftotal.loc[index,"KNNsNbr"]=len(nbours_index)
        dftotal.loc[index,"KSurvivalRate"]=dftotal.loc[nbours_index,"Survived"].mean()
    elif((ticket1!="LINE")&(ticket2==0)):
        dftotalindex=dftotal[(dftotal.Embarked==embarked)&(dftotal.Ticket2==0)&(dftotal.Ticket!="LINE")].copy()
        nbours_index=dftotalindex[np.abs(dftotalindex["Ticket1"].astype(int)-int(ticket1))<k].index
        dftotal.at[index,"KNNs"]=nbours_index.values
        dftotal.loc[index,"KNNsNbr"]=len(nbours_index)
        dftotal.loc[index,"KSurvivalRate"]=dftotalindex.loc[nbours_index,"Survived"].mean()
    else:
        dftotalindex=dftotal[(dftotal.Embarked==embarked)&(dftotal.Ticket1==ticket1)].copy()
        nbours_index=dftotalindex[np.abs(dftotalindex["Ticket2"].astype(int)-int(ticket2))<k].index
        dftotal.at[index,"KNNs"]=nbours_index.values
        dftotal.loc[index,"KNNsNbr"]=len(nbours_index)
        dftotal.loc[index,"KSurvivalRate"]=dftotalindex.loc[nbours_index,"Survived"].mean()

In [20]:
dftotal.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Ticket1,Ticket2,KNNs,KNNsNbr,KSurvivalRate
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,A/5,21171,"[0, 212, 227, 320, 1176]",5,0.000000
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,PC,17599,"[1, 30, 34, 64, 139, 155, 177, 273, 311, 375, ...",25,0.357143
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,STON/O2.,3101282,"[2, 142, 216, 403, 816]",5,0.600000
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,113803,0,"[3, 137, 270, 447, 555, 602, 607, 694, 724, 80...",13,0.500000
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,373450,0,[4],1,0.000000


In [80]:
(dftotal[(dftotal.KNNsNbr>5)&(dftotal.KSurvivalRate==0.0)
        &(dftotal.PassengerId>891)&(dftotal.Sex=="female")&(dftotal.Pclass==3)])


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Ticket1,Ticket2,KNNs,KNNsNbr,KSurvivalRate
924,NaN,NaN,S,23.4500,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",2,925,3,female,1,NaN,W./C. 6607,W./C.,6607,"[86, 147, 235, 436, 736, 783, 888, 924, 1058, ...",10,0.0
928,21.0,NaN,S,8.6625,"Cacic, Miss. Manda",0,929,3,female,0,NaN,315087,315087,0,"[163, 349, 404, 422, 471, 500, 534, 725, 844, ...",15,0.0
979,NaN,NaN,Q,7.7500,"O'Donoghue, Ms. Bridget",0,980,3,female,0,NaN,364856,364856,0,"[188, 428, 593, 657, 767, 979, 1173, 1279]",8,0.0
1031,10.0,NaN,S,46.9000,"Goodwin, Miss. Jessie Allis",2,1032,3,female,5,NaN,CA 2144,CA,2144,"[59, 71, 386, 480, 678, 683, 1030, 1031]",8,0.0
1044,36.0,NaN,S,12.1833,"Klasen, Mrs. (Hulda Kristina Eugenia Lofqvist)",2,1045,3,female,0,NaN,350405,350405,0,"[14, 175, 396, 1026, 1044, 1152, 1154, 1209, 1...",9,0.0
1050,26.0,NaN,S,13.7750,"Peacock, Mrs. Benjamin (Edith Nile)",2,1051,3,female,0,NaN,SOTON/O.Q. 3101315,SOTON/O.Q.,3101315,"[131, 210, 363, 465, 784, 1050, 1102, 1172, 11...",11,0.0
1060,22.0,NaN,S,8.9625,"Hellstrom, Miss. Hilda Maria",0,1061,3,female,0,NaN,7548,7548,0,"[40, 103, 474, 713, 882, 1060]",6,0.0
1079,NaN,NaN,S,69.5500,"Sage, Miss. Ada",2,1080,3,female,8,NaN,CA. 2343,CA.,2343,"[159, 180, 201, 324, 792, 846, 863, 1079, 1233...",11,0.0
1088,18.0,NaN,S,7.7750,"Nilsson, Miss. Berta Olivia",0,1089,3,female,0,NaN,347066,347066,0,"[129, 162, 231, 246, 276, 379, 406, 719, 764, ...",17,0.0
1154,1.0,NaN,S,12.1833,"Klasen, Miss. Gertrud Emilia",1,1155,3,female,1,NaN,350405,350405,0,"[14, 175, 396, 1026, 1044, 1152, 1154, 1209, 1...",9,0.0


In [81]:
(dftotal[(dftotal.KNNsNbr>5)&(dftotal.KSurvivalRate==0.0)
        &(dftotal.PassengerId>891)&(dftotal.Sex=="male")&(dftotal.Pclass==3)])


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Ticket1,Ticket2,KNNs,KNNsNbr,KSurvivalRate
896,14.00,NaN,S,9.2250,"Svensson, Mr. Johan Cervin",0,897,3,male,0,NaN,7538,7538,0,"[40, 103, 138, 713, 876, 896]",6,0.0
901,NaN,NaN,S,7.8958,"Ilieff, Mr. Ylio",0,902,3,male,0,NaN,349220,349220,0,"[29, 101, 321, 335, 384, 410, 601, 628, 650, 6...",18,0.0
947,35.00,NaN,S,7.8958,"Cor, Mr. Bartol",0,948,3,male,0,NaN,349230,349230,0,"[49, 223, 293, 294, 335, 353, 384, 410, 628, 6...",19,0.0
951,17.00,NaN,S,7.8958,"Dika, Mr. Mirko",0,952,3,male,0,NaN,349232,349232,0,"[49, 223, 293, 294, 335, 353, 384, 628, 646, 6...",17,0.0
962,21.00,NaN,S,7.8958,"Minkoff, Mr. Lazar",0,963,3,male,0,NaN,349211,349211,0,"[29, 76, 101, 105, 287, 321, 514, 566, 601, 66...",18,0.0
1025,43.00,NaN,S,7.8958,"Dintcheff, Mr. Valtcho",0,1026,3,male,0,NaN,349226,349226,0,"[223, 294, 321, 335, 384, 410, 628, 646, 650, ...",19,0.0
1030,40.00,NaN,S,46.9000,"Goodwin, Mr. Charles Frederick",6,1031,3,male,1,NaN,CA 2144,CA,2144,"[59, 71, 386, 480, 678, 683, 1030, 1031]",8,0.0
1036,31.00,NaN,S,18.0000,"Vander Planke, Mr. Julius",0,1037,3,male,3,NaN,345763,345763,0,"[18, 38, 80, 200, 333, 441, 873, 929, 1036, 1185]",10,0.0
1058,18.00,NaN,S,34.3750,"Ford, Mr. Edward Watson",2,1059,3,male,2,NaN,W./C. 6608,W./C.,6608,"[86, 147, 235, 436, 736, 783, 888, 924, 1058, ...",10,0.0
1102,NaN,NaN,S,7.0500,"Finoli, Mr. Luigi",0,1103,3,male,0,NaN,SOTON/O.Q. 3101308,SOTON/O.Q.,3101308,"[131, 210, 363, 465, 611, 784, 1050, 1102, 117...",12,0.0


# Shorting With Passenger Id

In [82]:
women_group =women_group_dead=dftotal[(dftotal.KNNsNbr>0)&(dftotal.KSurvivalRate==0.0)
        &(dftotal.PassengerId>891)&(dftotal.Sex=="female")&(dftotal.Pclass==3)]["PassengerId"].values
women_group_dead

array([ 925,  929,  955,  971,  980, 1017, 1032, 1045, 1051, 1061, 1080,
       1089, 1092, 1155, 1172, 1174, 1176, 1196, 1237, 1251, 1257, 1274,
       1300, 1301])

In [83]:
group = train.groupby(["Sex", "Pclass"])
group["Survived","Age","Fare"].agg(["mean","std","size"])

Survived                       Age                        Fare  \
                   mean       std size       mean        std size        mean   
Sex    Pclass                                                                   
female 1       0.968085  0.176716   94  34.611765  13.612052   94  106.125798   
       2       0.921053  0.271448   76  28.722973  12.872702   76   21.970121   
       3       0.500000  0.501745  144  21.750000  12.729964  144   16.118810   
male   1       0.368852  0.484484  122  41.281386  15.139570  122   67.226127   
       2       0.157407  0.365882  108  30.740707  14.793894  108   19.741782   
       3       0.135447  0.342694  347  26.507589  12.159514  347   12.661633   

                               
                     std size  
Sex    Pclass                  
female 1       74.259988   94  
       2       10.891796   76  
       3       11.690314  144  
male   1       77.548021  122  
       2       14.922235  108  
       3       11.681696  347

In [84]:
from sklearn.base import BaseEstimator

class DummyPrediction(BaseEstimator):
    def fit(self,X,Y=None):
        pass
    def predict(self,X):
        Y = X["Sex"] == "female"
        return Y.astype(int)

In [85]:
def print_scores(scores):
    print(scores)
    print(scores.mean())
    print(scores.std())

In [86]:
clf = DummyPrediction()
train["Survived_Model"] = clf.predict(train)

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet,SGDRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, precision_score,precision_recall_curve,recall_score,f1_score, roc_auc_score


In [87]:
#precision_score,precision_recall_curve,recall_score,f1_score, roc_auc_score
Y = train["Survived"]
Y_model = train["Survived_Model"]
print("recall score on training set",mean_squared_error(Y,Y_model))

recall score on training set 0.2132435465768799


In [88]:
Y = train["Survived"]
Y_model = train["Survived_Model"]
print("recall score on training set",recall_score(Y,Y_model))

recall score on training set 0.6812865497076024


In [89]:
scores=cross_validate(clf,train,train["Survived"],scoring=["f1","accuracy"],cv=20,return_train_score=False)

In [90]:
def display_cross_validate(scores):
    print("cross val scores")
    print("f1 scores", scores["test_f1"])
    print("f1 mean", scores["test_f1"].mean())
    print("f1 std", scores["test_f1"].std())
    print("accuracy scores", scores["test_accuracy"])
    print("accuracy mean", scores["test_accuracy"].mean())
    print("accuracy std", scores["test_accuracy"].std())

In [91]:
display_cross_validate(scores)

cross val scores
f1 scores [0.77272727 0.78787879 0.54545455 0.72727273 0.68571429 0.68571429
 0.73170732 0.90909091 0.75675676 0.66666667 0.69565217 0.74285714
 0.65       0.66666667 0.6        0.57142857 0.71428571 0.77419355
 0.56       0.75675676]
f1 mean 0.7000412064322199
f1 std 0.08591636169554233
accuracy scores [0.77777778 0.84444444 0.77777778 0.86666667 0.75555556 0.75555556
 0.75555556 0.91111111 0.8        0.71111111 0.84444444 0.79545455
 0.68181818 0.79545455 0.72727273 0.72727273 0.81818182 0.84090909
 0.75       0.79545455]
accuracy mean 0.7865909090909091
accuracy std 0.05525895919365663


In [92]:
scores_df =pd.DataFrame(list(scores.items()),columns=['Model','Score'])
scores_df.sort_values(['Score'])

AttributeError: 'NoneType' object has no attribute 'DataFrame'

In [93]:
print("roc_auc_score on training set",roc_auc_score(Y,Y_model))

roc_auc_score on training set 0.7668727830505226


In [94]:
test["Survived"]=clf.predict(test)

In [95]:
def process_name(df):
    df["FamilyName"] = df["Name"].map(lambda x: x.split(",")[0].strip())
    df["FullName"] = df["Name"].map(lambda x: x.split(",")[1].strip())
    df["Title"] = df["FullName"].map(lambda x: x.split(",")[0].strip())
    df["TwoLetters"] = df["FamilyName"].map(lambda x: x[-2:])
    
process_name(train)
process_name(test)

In [96]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_Model,FamilyName,FullName,Title,TwoLetters,TicketButlast
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Braund,Mr. Owen Harris,Mr. Owen Harris,nd,A/5 2117
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),Mrs. John Bradley (Florence Briggs Thayer),gs,PC 1759
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,Miss. Laina,Miss. Laina,en,STON/O2. 310128
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),Mrs. Jacques Heath (Lily May Peel),le,11380
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Allen,Mr. William Henry,Mr. William Henry,en,37345


In [97]:
train["Ticket"].value_counts()[:6]

347082      7
CA. 2343    7
1601        7
CA 2144     6
3101295     6
347088      6
Name: Ticket, dtype: int64

In [98]:
train["TicketButlast"] = train.Ticket.map(lambda x: x[:-1])
test["TicketButlast"] = test.Ticket.map(lambda x: x[:-1])

In [99]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_Model,FamilyName,FullName,Title,TwoLetters,TicketButlast
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Braund,Mr. Owen Harris,Mr. Owen Harris,nd,A/5 2117
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),Mrs. John Bradley (Florence Briggs Thayer),gs,PC 1759
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,Miss. Laina,Miss. Laina,en,STON/O2. 310128
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),Mrs. Jacques Heath (Lily May Peel),le,11380
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Allen,Mr. William Henry,Mr. William Henry,en,37345


In [100]:
train[train.TwoLetters =="ic"].sort_values("Ticket")[["PassengerId","Survived","Pclass","Name","Ticket"]]

,PassengerId,Survived,Pclass,Name,Ticket
158,159,0,3,"Smiljanic, Mr. Mile",315037
534,535,0,3,"Cacic, Miss. Marija",315084
500,501,0,3,"Calic, Mr. Petar",315086
349,350,0,3,"Dimic, Mr. Jovan",315088
471,472,0,3,"Cacic, Mr. Luka",315089
844,845,0,3,"Culumovic, Mr. Jeso",315090
163,164,0,3,"Calic, Mr. Jovo",315093
725,726,0,3,"Oreskovic, Mr. Luka",315094
404,405,0,3,"Oreskovic, Miss. Marija",315096
836,837,0,3,"Pasic, Mr. Jakob",315097


In [101]:
train[train.TwoLetters=="ic"].sort_values("Ticket")[["PassengerId","Survived","Pclass","Name","Ticket"]][:5]

,PassengerId,Survived,Pclass,Name,Ticket
158,159,0,3,"Smiljanic, Mr. Mile",315037
534,535,0,3,"Cacic, Miss. Marija",315084
500,501,0,3,"Calic, Mr. Petar",315086
349,350,0,3,"Dimic, Mr. Jovan",315088
471,472,0,3,"Cacic, Mr. Luka",315089


In [102]:
ticket_count_train = train["Ticket"].value_counts()
ticket_count_test = test["Ticket"].value_counts()
ticket_inter = np.intersect1d(train["Ticket"].values,test["Ticket"].values)

ticketButLast_count_train = train["TicketButlast"].value_counts()
ticketButLast_count_test = test["TicketButlast"].value_counts()
ticketButLast_inter = np.intersect1d(train["TicketButlast"].values,test["TicketButlast"].values)

In [103]:
for idx in train.index:
    ticket = train.loc[idx,"Ticket"]
    train.loc[idx,"CountTicket_InTrain"]=ticket_count_train[ticket]
    if(ticket in ticket_inter):
        train.loc[idx,"CountTicket"]=(ticket_count_train[ticket]+ticket_count_test[ticket])
    else:
        train.loc[idx,"CountTicket"]=ticket_count_train[ticket]

In [104]:
for idx in test.index:
    ticket = test.loc[idx,"Ticket"]
    test.loc[idx,"CountTicket_InTest"]=ticket_count_test[ticket]
    if(ticket in ticket_inter):
        test.loc[idx,"CountTicket"]=(ticket_count_train[ticket]+ticket_count_test[ticket])
    else:
        test.loc[idx,"CountTicket"]=ticket_count_test[ticket]

In [105]:
for idx in train.index:
    ticketButLast = train.loc[idx,"TicketButlast"]
    train.loc[idx,"CountTicketButLast_InTrain"]=ticketButLast_count_train[ticketButLast]
    if(ticketButLast in ticketButLast_inter):
        train.loc[idx,"CountTicketButLast"]=(ticketButLast_count_train[ticketButLast]
                                               +ticketButLast_count_test[ticketButLast])
    else:
        train.loc[idx,"CountTicketButLast"]=ticketButLast_count_train[ticketButLast]


In [106]:
for idx in test.index:
    ticketButLast = test.loc[idx,"TicketButlast"]
    test.loc[idx,"CountTicketButLast_InTest"]=ticketButLast_count_test[ticketButLast]
    if(ticketButLast in ticketButLast_inter):
        test.loc[idx,"CountTicketButLast"]=(ticketButLast_count_train[ticketButLast]
                                               +ticketButLast_count_test[ticketButLast])
        test.loc[idx,"CountTicketButLast_InTrain"]=(ticketButLast_count_train[ticketButLast])
    else:
        test.loc[idx,"CountTicketButLast"]=ticketButLast_count_test[ticketButLast]

In [107]:
for idx in train.index:
    ticket = train.loc[idx,"Ticket"]
    if(ticket in ticket_inter):
        train.loc[idx,"FareCorrect"]=train.loc[idx,"Fare"]/(ticket_count_train[ticket]
                                                                +ticket_count_test[ticket])
    else:
        train.loc[idx,"FareCorrect"]=train.loc[idx,"Fare"]/(ticket_count_train[ticket])

In [108]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,Braund,Mr. Owen Harris,Mr. Owen Harris,nd,A/5 2117,1.0,1.0,4.0,5.0,7.25000
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,Cumings,Mrs. John Bradley (Florence Briggs Thayer),Mrs. John Bradley (Florence Briggs Thayer),gs,PC 1759,1.0,2.0,8.0,14.0,35.64165
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,Heikkinen,Miss. Laina,Miss. Laina,en,STON/O2. 310128,1.0,1.0,2.0,2.0,7.92500
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,Futrelle,Mrs. Jacques Heath (Lily May Peel),Mrs. Jacques Heath (Lily May Peel),le,11380,2.0,2.0,7.0,8.0,26.55000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,Allen,Mr. William Henry,Mr. William Henry,en,37345,1.0,1.0,1.0,1.0,8.05000


In [109]:
print("Example of correction for large Pclass 3 Family")
train[train.FamilyName=="Panula"][["Survived","Pclass","Name","Ticket","Fare","FareCorrect"]]

Example of correction for large Pclass 3 Family


,Survived,Pclass,Name,Ticket,Fare,FareCorrect
50,0,3,"Panula, Master. Juha Niilo",3101295,39.6875,5.669643
164,0,3,"Panula, Master. Eino Viljami",3101295,39.6875,5.669643
266,0,3,"Panula, Mr. Ernesti Arvid",3101295,39.6875,5.669643
638,0,3,"Panula, Mrs. Juha (Maria Emilia Ojala)",3101295,39.6875,5.669643
686,0,3,"Panula, Mr. Jaako Arnold",3101295,39.6875,5.669643
824,0,3,"Panula, Master. Urho Abraham",3101295,39.6875,5.669643


In [110]:
dfFamilytest=test[(test["Parch"]>0)&(test["Sex"]=="female")]
familiestest=dfFamilytest["FamilyName"]

In [111]:
dfFamilyTrain=train[(train["Parch"]>0)&(train["Survived"]==0)]
dfFamily=dfFamilyTrain[(dfFamilyTrain["Sex"]=="female")|(dfFamilyTrain["Age"]<10)]
familiestrain=dfFamily["FamilyName"]


In [112]:
train[(train["PassengerId"]==159)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
158,159,0,3,"Smiljanic, Mr. Mile",male,NaN,0,0,315037,8.6625,...,Smiljanic,Mr. Mile,Mr. Mile,ic,31503,1.0,1.0,1.0,1.0,8.6625


In [113]:
print("Women who have family members who died in the training set.")
intersection=np.intersect1d(familiestest,familiestrain)
intersection

Women who have family members who died in the training set.


array(['Andersson', 'Goodwin', 'Johnston', 'Lefebre', 'Rosblom', 'Sage'],
      dtype=object)

In [114]:
women_died_ids=dfFamilytest[dfFamilytest["FamilyName"].isin(intersection)]["PassengerId"].index

In [115]:
col_index=test.columns.get_loc("Survived")
test.iloc[women_died_ids,col_index]=0

In [116]:
print("Mothers and Sisters who died with their families")
test.iloc[women_died_ids,:]

Mothers and Sisters who died with their families


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,...,Johnston,"Mrs. Andrew G (Elizabeth Lily"" Watson)""","Mrs. Andrew G (Elizabeth Lily"" Watson)""",on,W./C. 660,2.0,4.0,3.0,10.0,7.0
132,1024,3,"Lefebre, Mrs. Frank (Frances)",female,NaN,0,4,4133,25.4667,NaN,...,Lefebre,Mrs. Frank (Frances),Mrs. Frank (Frances),re,413,1.0,5.0,1.0,10.0,9.0
140,1032,3,"Goodwin, Miss. Jessie Allis",female,10.0,5,2,CA 2144,46.9000,NaN,...,Goodwin,Miss. Jessie Allis,Miss. Jessie Allis,in,CA 214,2.0,8.0,2.0,8.0,6.0
188,1080,3,"Sage, Miss. Ada",female,NaN,8,2,CA. 2343,69.5500,NaN,...,Sage,Miss. Ada,Miss. Ada,ge,CA. 234,4.0,11.0,4.0,11.0,7.0
214,1106,3,"Andersson, Miss. Ida Augusta Margareta",female,38.0,4,2,347091,7.7750,NaN,...,Andersson,Miss. Ida Augusta Margareta,Miss. Ida Augusta Margareta,on,34709,1.0,1.0,2.0,2.0,NaN
284,1176,3,"Rosblom, Miss. Salli Helena",female,2.0,1,1,370129,20.2125,NaN,...,Rosblom,Miss. Salli Helena,Miss. Salli Helena,om,37012,1.0,3.0,1.0,3.0,2.0
365,1257,3,"Sage, Mrs. John (Annie Bullen)",female,NaN,1,9,CA. 2343,69.5500,NaN,...,Sage,Mrs. John (Annie Bullen),Mrs. John (Annie Bullen),ge,CA. 234,4.0,11.0,4.0,11.0,7.0


In [117]:
test[(test.Title=="Master")&(test.Survived==0)]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain


In [118]:
print("Male passenger travelling with Olsen, Master. Artur Karl who survived:")
train[train.Ticket.map(lambda x: x[:-1])=="C 1736"]

Male passenger travelling with Olsen, Master. Artur Karl who survived:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
127,128,1,3,"Madsen, Mr. Fridtjof Arne",male,24.0,0,0,C 17369,7.1417,...,Madsen,Mr. Fridtjof Arne,Mr. Fridtjof Arne,en,C 1736,1.0,1.0,1.0,2.0,7.1417


In [119]:
test[test.FamilyName=="Peacock"]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
159,1051,3,"Peacock, Mrs. Benjamin (Edith Nile)",female,26.00,0,2,SOTON/O.Q. 3101315,13.775,NaN,...,Peacock,Mrs. Benjamin (Edith Nile),Mrs. Benjamin (Edith Nile),ck,SOTON/O.Q. 310131,3.0,3.0,4.0,7.0,3.0
281,1173,3,"Peacock, Master. Alfred Edward",male,0.75,1,1,SOTON/O.Q. 3101315,13.775,NaN,...,Peacock,Master. Alfred Edward,Master. Alfred Edward,ck,SOTON/O.Q. 310131,3.0,3.0,4.0,7.0,3.0
409,1301,3,"Peacock, Miss. Treasteall",female,3.00,1,1,SOTON/O.Q. 3101315,13.775,NaN,...,Peacock,Miss. Treasteall,Miss. Treasteall,ck,SOTON/O.Q. 310131,3.0,3.0,4.0,7.0,3.0


In [120]:
olsen_idx=test[(test.FamilyName=="Olsen")].index
test.loc[olsen_idx,"Survived"]=1
peacock_idx=test[(test.FamilyName=="Peacock")].index
test.loc[peacock_idx,"Survived"]=0

In [121]:
print("Panula family Ticket number 3101295")
train[train.Ticket=="3101295"]

Panula family Ticket number 3101295


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
50,51,0,3,"Panula, Master. Juha Niilo",male,7.0,4,1,3101295,39.6875,...,Panula,Master. Juha Niilo,Master. Juha Niilo,la,310129,6.0,7.0,8.0,11.0,5.669643
164,165,0,3,"Panula, Master. Eino Viljami",male,1.0,4,1,3101295,39.6875,...,Panula,Master. Eino Viljami,Master. Eino Viljami,la,310129,6.0,7.0,8.0,11.0,5.669643
266,267,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,3101295,39.6875,...,Panula,Mr. Ernesti Arvid,Mr. Ernesti Arvid,la,310129,6.0,7.0,8.0,11.0,5.669643
638,639,0,3,"Panula, Mrs. Juha (Maria Emilia Ojala)",female,41.0,0,5,3101295,39.6875,...,Panula,Mrs. Juha (Maria Emilia Ojala),Mrs. Juha (Maria Emilia Ojala),la,310129,6.0,7.0,8.0,11.0,5.669643
686,687,0,3,"Panula, Mr. Jaako Arnold",male,14.0,4,1,3101295,39.6875,...,Panula,Mr. Jaako Arnold,Mr. Jaako Arnold,la,310129,6.0,7.0,8.0,11.0,5.669643
824,825,0,3,"Panula, Master. Urho Abraham",male,2.0,4,1,3101295,39.6875,...,Panula,Master. Urho Abraham,Master. Urho Abraham,la,310129,6.0,7.0,8.0,11.0,5.669643


In [122]:
print("Travelling with them in test set")
test[test.Ticket=="3101295"]

Travelling with them in test set


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
367,1259,3,"Riihivouri, Miss. Susanna Juhantytar Sanni""""",female,22.0,0,0,3101295,39.6875,NaN,...,Riihivouri,"Miss. Susanna Juhantytar Sanni""""","Miss. Susanna Juhantytar Sanni""""",ri,310129,1.0,7.0,3.0,11.0,8.0


In [123]:
print("Most occuring ticket in train set, a group of Chinese. Most of them survived.")
train[train.Ticket=="1601"]

Most occuring ticket in train set, a group of Chinese. Most of them survived.


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
74,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,...,Bing,Mr. Lee,Mr. Lee,ng,160,7.0,8.0,7.0,8.0,7.061975
169,170,0,3,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,...,Ling,Mr. Lee,Mr. Lee,ng,160,7.0,8.0,7.0,8.0,7.061975
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,...,Lang,Mr. Fang,Mr. Fang,ng,160,7.0,8.0,7.0,8.0,7.061975
643,644,1,3,"Foo, Mr. Choong",male,NaN,0,0,1601,56.4958,...,Foo,Mr. Choong,Mr. Choong,oo,160,7.0,8.0,7.0,8.0,7.061975
692,693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,...,Lam,Mr. Ali,Mr. Ali,am,160,7.0,8.0,7.0,8.0,7.061975
826,827,0,3,"Lam, Mr. Len",male,NaN,0,0,1601,56.4958,...,Lam,Mr. Len,Mr. Len,am,160,7.0,8.0,7.0,8.0,7.061975
838,839,1,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,...,Chip,Mr. Chang,Mr. Chang,ip,160,7.0,8.0,7.0,8.0,7.061975


In [124]:
print("accompanying them in test set : ")
test[test.Ticket=="1601"]

accompanying them in test set : 


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,...,Hee,Mr. Ling,Mr. Ling,ee,160,1.0,8.0,1.0,8.0,7.0


In [125]:
most_occ_ticks=train["Ticket"].value_counts()[:19]

In [126]:
df=test[test["Ticket"].isin(most_occ_ticks.index)].sort_values("Ticket")
df_women=df[df["Sex"]=="female"]

In [127]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [182]:
demo = pd.concat([train,test], axis=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [184]:
demo.head()

,Age,Cabin,CountTicket,CountTicketButLast,CountTicketButLast_InTest,CountTicketButLast_InTrain,CountTicket_InTest,CountTicket_InTrain,Embarked,FamilyName,...,PassengerId,Pclass,Sex,SibSp,Survived,Survived_Model,Ticket,TicketButlast,Title,TwoLetters
0,22.0,NaN,1.0,5.0,NaN,4.0,NaN,1.0,S,Braund,...,1,3,male,1,0,0.0,A/5 21171,A/5 2117,Mr. Owen Harris,nd
1,38.0,C85,2.0,14.0,NaN,8.0,NaN,1.0,C,Cumings,...,2,1,female,1,1,1.0,PC 17599,PC 1759,Mrs. John Bradley (Florence Briggs Thayer),gs
2,26.0,NaN,1.0,2.0,NaN,2.0,NaN,1.0,S,Heikkinen,...,3,3,female,0,1,1.0,STON/O2. 3101282,STON/O2. 310128,Miss. Laina,en
3,35.0,C123,2.0,8.0,NaN,7.0,NaN,2.0,S,Futrelle,...,4,1,female,1,1,1.0,113803,11380,Mrs. Jacques Heath (Lily May Peel),le
4,35.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,S,Allen,...,5,3,male,0,0,0.0,373450,37345,Mr. William Henry,en


In [183]:
for dataset in demo:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

AttributeError: 'str' object has no attribute 'Name'

In [ ]:
demo.Sex.value_counts()

In [ ]:
print("Women who died with their groups. We already spotted some mothers above.")
for idx in df_women.index:
    ticket = df_women.loc[idx,"Ticket"]
    nbr_survivors = train[train["Ticket"]==ticket]["Survived"].sum()
    if(nbr_survivors==0):
        print(df_women.loc[idx,"Name"])
        test.loc[idx,"Survived"]=0

In [ ]:
df_men=df[(df["Sex"]=="male")&(df["SibSp"]==0)&(df["Parch"]==0)]
df_men

In [ ]:
print("Man who survived with the majority of his group:")
for idx in df_men.index:
    ticket = df_men.loc[idx,"Ticket"]
    mean_survivors = train[train["Ticket"]==ticket]["Survived"].mean()
    if(mean_survivors>0.5):
        print(df_men.loc[idx,"Name"])
        test.loc[idx,"Survived"]=1

In [ ]:
test[(test.CountTicketButLast>10)&(test.Pclass==3)&(test.SibSp==0)&(test.Parch==0)&(test.Sex=="female")&
      (test.Title=="Miss")]

In [ ]:
print("Passengers of train set who travelled with Nieminen, Miss. Manta Josefina")
train[train.TicketButlast=="310129"]

In [ ]:
print("Passengers of train set who travelled with Nieminen, Miss. Jenny Lovisa")
train[train.TicketButlast=="34708"]

In [129]:
idx=test[(test.CountTicketButLast>10)&(test.Pclass==3)&(test.SibSp==0)&(test.Parch==0)&(test.Sex=="female")&
      (test.Title=="Miss")].index
test.loc[idx,"Survived"]=0

In [130]:
print("People called ---ic died :")
train[train.TwoLetters=="ic"] ## FF

People called ---ic died :


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTrain,CountTicket,CountTicketButLast_InTrain,CountTicketButLast,FareCorrect
108,109,0,3,"Rekic, Mr. Tido",male,38.0,0,0,349249,7.8958,...,Rekic,Mr. Tido,Mr. Tido,ic,34924,1.0,1.0,10.0,10.0,7.8958
130,131,0,3,"Drazenoic, Mr. Jozef",male,33.0,0,0,349241,7.8958,...,Drazenoic,Mr. Jozef,Mr. Jozef,ic,34924,1.0,1.0,10.0,10.0,7.8958
158,159,0,3,"Smiljanic, Mr. Mile",male,NaN,0,0,315037,8.6625,...,Smiljanic,Mr. Mile,Mr. Mile,ic,31503,1.0,1.0,1.0,1.0,8.6625
163,164,0,3,"Calic, Mr. Jovo",male,17.0,0,0,315093,8.6625,...,Calic,Mr. Jovo,Mr. Jovo,ic,31509,1.0,1.0,6.0,9.0,8.6625
285,286,0,3,"Stankovic, Mr. Ivan",male,33.0,0,0,349239,8.6625,...,Stankovic,Mr. Ivan,Mr. Ivan,ic,34923,1.0,1.0,7.0,11.0,8.6625
313,314,0,3,"Hendekovic, Mr. Ignjac",male,28.0,0,0,349243,7.8958,...,Hendekovic,Mr. Ignjac,Mr. Ignjac,ic,34924,1.0,1.0,10.0,10.0,7.8958
349,350,0,3,"Dimic, Mr. Jovan",male,42.0,0,0,315088,8.6625,...,Dimic,Mr. Jovan,Mr. Jovan,ic,31508,1.0,1.0,5.0,8.0,8.6625
404,405,0,3,"Oreskovic, Miss. Marija",female,20.0,0,0,315096,8.6625,...,Oreskovic,Miss. Marija,Miss. Marija,ic,31509,1.0,1.0,6.0,9.0,8.6625
471,472,0,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,...,Cacic,Mr. Luka,Mr. Luka,ic,31508,1.0,1.0,5.0,8.0,8.6625
500,501,0,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.6625,...,Calic,Mr. Petar,Mr. Petar,ic,31508,1.0,1.0,5.0,8.0,8.6625


In [131]:
test[(test.TwoLetters.isin(["ic","ff"]))&(test.Sex=="female")]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
37,929,3,"Cacic, Miss. Manda",female,21.0,0,0,315087,8.6625,NaN,...,Cacic,Miss. Manda,Miss. Manda,ic,31508,1.0,1.0,3.0,8.0,5.0
280,1172,3,"Oreskovic, Miss. Jelka",female,23.0,0,0,315085,8.6625,NaN,...,Oreskovic,Miss. Jelka,Miss. Jelka,ic,31508,1.0,1.0,3.0,8.0,5.0


In [132]:
idx=test[(test.TwoLetters.isin(["ic","ff"]))&(test.Sex=="female")].index
test.loc[idx,"Survived"]=0

In [133]:
idx

Int64Index([37, 280], dtype='int64')

In [134]:
test[test["PassengerId"]==1297]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
405,1297,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.0,0,0,SC/PARIS 2166,13.8625,D38,...,Nourney,"Mr. Alfred (Baron von Drachstedt"")""","Mr. Alfred (Baron von Drachstedt"")""",ey,SC/PARIS 216,1.0,1.0,3.0,4.0,1.0


# ALGORITHMIC CLASSIFICATION for SINGLE WOMEN

In [135]:
single_women = train[(train["Sex"]=="female")&(train["SibSp"]==0)&(train["Parch"]==0)]
single_women["Age"]=single_women["Age"].fillna(single_women["Age"].median())
features=["Age","Pclass","Fare"]
x_single_women = single_women[features]
y_single_women = single_women["Survived"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [136]:
xg = GradientBoostingClassifier()
kfold = StratifiedKFold(10)

In [137]:
param_grid={
    'n_estimators':[100,200,300],
    'min_samples_split':[2,3,4],
    'max_depth' : np.arange(3,7,1)
}


In [138]:
grid = GridSearchCV(xg,param_grid=param_grid,cv=kfold,scoring="accuracy")
grid.fit(x_single_women,y_single_women)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 300], 'min_samples_split': [2, 3, 4], 'max_depth': array([3, 4, 5, 6])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [139]:
grid.best_params_

{'max_depth': 4, 'min_samples_split': 3, 'n_estimators': 200}

In [140]:
test_df=test[(test["Sex"]=="female")&(test["SibSp"]==0)&(test["Parch"]==0)]
test_df["Age"]=test_df["Age"].fillna(test_df["Age"].median())
test_df["Fare"]=test_df["Fare"].fillna(test_df["Fare"].median())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [141]:
Y_scores_pclass3 = grid.best_estimator_.predict_proba(test_df[features])[:,1]
test_df["Y_scores"]=Y_scores_pclass3

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [142]:
print("Five worst scores of xgboost :")
test_df.sort_values("Y_scores",ascending=True)[:5]

Five worst scores of xgboost :


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain,Y_scores
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,...,Miss. Kate,Miss. Kate,ly,33097,1.0,1.0,2.0,3.0,1.0,0.000012
313,1205,3,"Carr, Miss. Jeannie",female,37.0,0,0,368364,7.7500,NaN,...,Miss. Jeannie,Miss. Jeannie,rr,36836,1.0,1.0,1.0,1.0,NaN,0.000075
291,1183,3,"Daly, Miss. Margaret Marcella Maggie""""",female,30.0,0,0,382650,6.9500,NaN,...,"Miss. Margaret Marcella Maggie""""","Miss. Margaret Marcella Maggie""""",ly,38265,1.0,1.0,3.0,9.0,6.0,0.000336
138,1030,3,"Drapkin, Miss. Jennie",female,23.0,0,0,SOTON/OQ 392083,8.0500,NaN,...,Miss. Jennie,Miss. Jennie,in,SOTON/OQ 39208,1.0,1.0,1.0,4.0,3.0,0.000925
280,1172,3,"Oreskovic, Miss. Jelka",female,23.0,0,0,315085,8.6625,NaN,...,Miss. Jelka,Miss. Jelka,ic,31508,1.0,1.0,3.0,8.0,5.0,0.001014


# ISOLATED WOMEN

In [143]:
single_poor_wmn=test[(test.CountTicketButLast<=2)&(test.Sex=="female")
                       &(test.Pclass==3)&(test.SibSp==0)&(test.Parch==0)].index
test.loc[single_poor_wmn,"Survived"]=0

In [144]:
print("Single poor isolated women with no family")
test.loc[single_poor_wmn,:]

Single poor isolated women with no family


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,...,Roth,Miss. Sarah A,Miss. Sarah A,th,34271,1.0,1.0,1.0,1.0,NaN
66,958,3,"Burns, Miss. Mary Delia",female,18.0,0,0,330963,7.8792,NaN,...,Burns,Miss. Mary Delia,Miss. Mary Delia,ns,33096,1.0,1.0,2.0,2.0,NaN
79,971,3,"Doyle, Miss. Elizabeth",female,24.0,0,0,368702,7.7500,NaN,...,Doyle,Miss. Elizabeth,Miss. Elizabeth,le,36870,1.0,1.0,1.0,2.0,1.0
86,978,3,"Barry, Miss. Julia",female,27.0,0,0,330844,7.8792,NaN,...,Barry,Miss. Julia,Miss. Julia,ry,33084,1.0,1.0,1.0,1.0,NaN
87,979,3,"Badman, Miss. Emily Louisa",female,18.0,0,0,A/4 31416,8.0500,NaN,...,Badman,Miss. Emily Louisa,Miss. Emily Louisa,an,A/4 3141,1.0,1.0,1.0,1.0,NaN
98,990,3,"Braf, Miss. Elin Ester Maria",female,20.0,0,0,347471,7.8542,NaN,...,Braf,Miss. Elin Ester Maria,Miss. Elin Ester Maria,af,34747,1.0,1.0,1.0,2.0,1.0
111,1003,3,"Shine, Miss. Ellen Natalia",female,NaN,0,0,330968,7.7792,NaN,...,Shine,Miss. Ellen Natalia,Miss. Ellen Natalia,ne,33096,1.0,1.0,2.0,2.0,NaN
113,1005,3,"Buckley, Miss. Katherine",female,18.5,0,0,329944,7.2833,NaN,...,Buckley,Miss. Katherine,Miss. Katherine,ey,32994,1.0,1.0,1.0,1.0,NaN
160,1052,3,"Smyth, Miss. Julia",female,NaN,0,0,335432,7.7333,NaN,...,Smyth,Miss. Julia,Miss. Julia,th,33543,1.0,1.0,1.0,1.0,NaN
200,1092,3,"Murphy, Miss. Nora",female,NaN,0,0,36568,15.5000,NaN,...,Murphy,Miss. Nora,Miss. Nora,hy,3656,1.0,2.0,1.0,2.0,1.0


In [145]:
idx=test[test.PassengerId.isin(women_group_dead)].index
test.loc[idx,"Survived"]=0

In [146]:
print("Men who are predicted to survive")
test[(test["Survived"]==1)& (test["Sex"]=="male")]

Men who are predicted to survive


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
21,913,3,"Olsen, Master. Artur Karl",male,9.0,0,1,C 17368,3.1708,NaN,...,Olsen,Master. Artur Karl,Master. Artur Karl,en,C 1736,1.0,1.0,1.0,2.0,1.0


In [147]:
print("WOMEN who are predicted to survive")
test[(test["Survived"]==0)& (test["Sex"]=="female")]

WOMEN who are predicted to survive


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,...,Johnston,"Mrs. Andrew G (Elizabeth Lily"" Watson)""","Mrs. Andrew G (Elizabeth Lily"" Watson)""",on,W./C. 660,2.0,4.0,3.0,10.0,7.0
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,...,Roth,Miss. Sarah A,Miss. Sarah A,th,34271,1.0,1.0,1.0,1.0,NaN
37,929,3,"Cacic, Miss. Manda",female,21.0,0,0,315087,8.6625,NaN,...,Cacic,Miss. Manda,Miss. Manda,ic,31508,1.0,1.0,3.0,8.0,5.0
63,955,3,"Bradley, Miss. Bridget Delia",female,22.0,0,0,334914,7.7250,NaN,...,Bradley,Miss. Bridget Delia,Miss. Bridget Delia,ey,33491,1.0,1.0,2.0,3.0,1.0
66,958,3,"Burns, Miss. Mary Delia",female,18.0,0,0,330963,7.8792,NaN,...,Burns,Miss. Mary Delia,Miss. Mary Delia,ns,33096,1.0,1.0,2.0,2.0,NaN
79,971,3,"Doyle, Miss. Elizabeth",female,24.0,0,0,368702,7.7500,NaN,...,Doyle,Miss. Elizabeth,Miss. Elizabeth,le,36870,1.0,1.0,1.0,2.0,1.0
86,978,3,"Barry, Miss. Julia",female,27.0,0,0,330844,7.8792,NaN,...,Barry,Miss. Julia,Miss. Julia,ry,33084,1.0,1.0,1.0,1.0,NaN
87,979,3,"Badman, Miss. Emily Louisa",female,18.0,0,0,A/4 31416,8.0500,NaN,...,Badman,Miss. Emily Louisa,Miss. Emily Louisa,an,A/4 3141,1.0,1.0,1.0,1.0,NaN
88,980,3,"O'Donoghue, Ms. Bridget",female,NaN,0,0,364856,7.7500,NaN,...,O'Donoghue,Ms. Bridget,Ms. Bridget,ue,36485,1.0,1.0,3.0,5.0,2.0
98,990,3,"Braf, Miss. Elin Ester Maria",female,20.0,0,0,347471,7.8542,NaN,...,Braf,Miss. Elin Ester Maria,Miss. Elin Ester Maria,af,34747,1.0,1.0,1.0,2.0,1.0


# CSV OUTPUT

In [175]:
test_sorted=test.sort_values("PassengerId")
test_sorted=test_sorted[["PassengerId","Survived"]]

In [176]:
test_sorted.to_csv("pitanic.csv",index=False)
print('print csv')

print csv


In [180]:
lol = pd.read_csv("pitanic.csv")
lol1 = lol.set_index("PassengerId")
lol1

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1


In [181]:
lol.to_csv("pitani1c.csv",index=False)
print('print csv')

print csv


In [157]:
import pandas as pd
data = pd.read_csv("input/Titanic.csv")
data.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived', 'FamilyName',
       'FullName', 'Title', 'TwoLetters', 'TicketButlast',
       'CountTicket_InTest', 'CountTicket', 'CountTicketButLast_InTest',
       'CountTicketButLast', 'CountTicketButLast_InTrain'],
      dtype='object')

In [158]:
#df.drop(columns=['B', 'C'])
df.drop(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilyName',
       'FullName', 'Title', 'TwoLetters', 'TicketButlast',
       'CountTicket_InTest', 'CountTicket', 'CountTicketButLast_InTest',
       'CountTicketButLast', 'CountTicketButLast_InTrain'], axis=1)

,PassengerId,Survived
306,1198,0
141,1033,1
39,931,0
53,945,1
69,961,1
367,1259,1
379,1271,0
154,1046,0
174,1066,0
389,1281,0


In [159]:
data1 = pd.read_csv("finally.csv")


In [165]:
data1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FamilyName,FullName,Title,TwoLetters,TicketButlast,CountTicket_InTest,CountTicket,CountTicketButLast_InTest,CountTicketButLast,CountTicketButLast_InTrain
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,...,Kelly,Mr. James,Mr. James,ly,33091,1.0,1.0,2.0,3.0,1.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,...,Wilkes,Mrs. James (Ellen Needs),Mrs. James (Ellen Needs),es,36327,1.0,1.0,1.0,1.0,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,...,Myles,Mr. Thomas Francis,Mr. Thomas Francis,es,24027,1.0,1.0,1.0,1.0,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,...,Wirz,Mr. Albert,Mr. Albert,rz,31515,1.0,1.0,4.0,6.0,2.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,...,Hirvonen,Mrs. Alexander (Helga E Lindqvist),Mrs. Alexander (Helga E Lindqvist),en,310129,1.0,2.0,3.0,11.0,8.0
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,...,Svensson,Mr. Johan Cervin,Mr. Johan Cervin,on,753,1.0,1.0,1.0,3.0,2.0
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,...,Connolly,Miss. Kate,Miss. Kate,ly,33097,1.0,1.0,2.0,3.0,1.0
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,...,Caldwell,Mr. Albert Francis,Mr. Albert Francis,ll,24873,1.0,3.0,2.0,6.0,4.0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,...,Abrahim,Mrs. Joseph (Sophie Halaut Easu),Mrs. Joseph (Sophie Halaut Easu),im,265,1.0,1.0,9.0,16.0,7.0
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,...,Davies,Mr. John Samuel,Mr. John Samuel,es,A/4 4887,1.0,3.0,2.0,4.0,2.0


In [166]:
data1.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived', 'FamilyName',
       'FullName', 'Title', 'TwoLetters', 'TicketButlast',
       'CountTicket_InTest', 'CountTicket', 'CountTicketButLast_InTest',
       'CountTicketButLast', 'CountTicketButLast_InTrain'],
      dtype='object')

In [168]:
data2 = data1.drop(columns = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilyName',
       'FullName', 'Title', 'TwoLetters', 'TicketButlast',
       'CountTicket_InTest', 'CountTicket', 'CountTicketButLast_InTest',
       'CountTicketButLast', 'CountTicketButLast_InTrain'], )

In [169]:
data3 = data2.set_index("PassengerId")

In [170]:
data3

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1


In [ ]:
data3.to_csv('final.csv')

In [ ]:
test.Survived